In [22]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [24]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import time
import pandas as pd
import numpy as np
import sys
from io import StringIO
import glob
if not hasattr(sys, "version_info") or sys.version_info < (3, 5):
  raise SystemExit("This program requires Python 3.5 or later.")
import csv
import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import os
## https://github.com/ipython/ipyparallel
import ipyparallel as ipp
from IPython.display import JSON
import json
import statistics as st
from pympler import asizeof
from hurry.filesize import size
data_sources = glob.glob("data/*_shops.xml")
plt.rcParams['figure.figsize'] = [30/2.54, 20/2.54]
tmpFolder = "%s/tmp" % os.getcwd()
import tabletext
import sqlite3
# tags
client = ipp.Client()
workers = client[:]
print("engines count: %d" % len(client.ids))
print("source files count: %d" % len(data_sources))

engines count: 4
source files count: 1320


In [25]:
def mstime():
    import time
    return round(time.time() * 1000)

def sync():
    import os
    os.system('sync')
    
def touch(path):
    import os
    with open(path, 'a'):
        os.utime(path, None)
    sync()
    
def unic(l: list) -> list :
    return list(set(l))

def cleanupTmp():
    print("clean tmp in %s" % tmpFolder)
    for filename in os.listdir(tmpFolder):
        file_path = os.path.join(tmpFolder, filename)
        print("remove %s" % file_path)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [26]:
workers["tmpFolder"] = tmpFolder
workers["dbquery_cratetable"] = '''
CREATE TABLE "ttids" (
	"rowid"	INTEGER NOT NULL UNIQUE,
	"shapshotid"	TEXT NOT NULL,
	"ttid"	INTEGER NOT NULL,
	"buy_list"	TEXT NOT NULL,
	"sell_list"	TEXT NOT NULL,
	"capacity"	INTEGER NOT NULL,
	"buystat_avg"	REAL NOT NULL,
	"buystat_min"	REAL NOT NULL,
	"buystat_max"	REAL NOT NULL,
	"buystat_median"	REAL NOT NULL,
	"buystat_gap"	REAL NOT NULL,
	"buystat_stdev"	REAL NOT NULL,
	"sellstat_avg"	REAL NOT NULL,
	"sellstat_min"	REAL NOT NULL,
	"sellstat_max"	REAL NOT NULL,
	"sellstat_median"	REAL NOT NULL,
	"sellstat_gap"	REAL NOT NULL,
	"sellstat_stdev"	REAL NOT NULL,
	PRIMARY KEY("rowid" AUTOINCREMENT)
);'''
dbquery_cratetable = workers["dbquery_cratetable"][0]
workers["dbquery_insert"] = '''
INSERT INTO 'ttids'(
"shapshotid",
"ttid",
"buy_list",
"sell_list",
"capacity",
"buystat_avg",
"buystat_min",
"buystat_max",
"buystat_median",
"buystat_gap",
"buystat_stdev",
"sellstat_avg",
"sellstat_min",
"sellstat_max",
"sellstat_median",
"sellstat_gap",
"sellstat_stdev"
) VALUES (
?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'''
dbquery_insert = workers["dbquery_insert"][0]

In [27]:
def processSnapshot(fname):
    import xml.etree.ElementTree as ET
    import statistics as st
    import sqlite3
    import os
    import time
    from pathlib import Path

    def mstime():
        return round(time.time() * 1000)
    def sync():
        os.system('sync')
    def touch(path):
        with open(path, 'a'):
            os.utime(path, None)
        sync()
    
    log = []
    p = Path(fname)
    baseDir = str(p.parent)
    dbName = "%s/%s.%s.db" % (tmpFolder, str(p.name), mstime())
    log.append("process %s. base: %s, tmp database: %s" % (fname, baseDir, dbName))
    def processShop(tree, time):
        shopData = []
        ttidList = []
        for good in tree:
            goodData = {}
            goodData["ttid"] = int(good.findall("ttid")[0].text)
            ttidList.append(goodData["ttid"])
            goodData["time"] = time
            goodData["shop"] = tree.attrib["name"]
            goodData["shopid"] = tree.attrib["id"]
            goodData["shopmoney"] = tree.attrib["money"]
            sellElems = good.findall("sell")
            if sellElems:
                goodData["sell"] = float(sellElems[0].text)
            else:
                goodData["sell"] = 0.0

            buyElems = good.findall("buy")
            if buyElems:
                goodData["buy"] = float(buyElems[0].text)
            else:
                goodData["buy"] = 0.0
            goodData["count"] = int(good.findall("count")[0].text)
            goodData["thingname"] = good.findall("thingname")[0].text
            shopData.append(goodData)
        return (shopData, ttidList)
    

    if os.path.exists(dbName):
        os.remove(dbName)
    sync()
    touch(dbName)
    timetag = "lastupdatetime"
    shopTag = "shop"
    xmltree = ET.parse(fname)
    xmlroot = xmltree.getroot()
    timeelem = xmlroot.find(timetag)
    dataTime = int(timeelem.text)
    orders = []
    ttids = []
    con = sqlite3.connect(dbName)    
    cur = con.cursor()
    q = dbquery_cratetable.replace("\n", " ")
    cur.execute(q)
    for shop in xmlroot.findall(shopTag):
        rc = processShop(shop, dataTime)
        shopData = rc[0]
        ttidList = rc[1]
        orders.extend(shopData)
        ttids.extend(ttidList)
    ttid_summary = []
    for ttid in ttids:
        ttid_stat = {}
        ttid_stat["ttid"] = ttid
        ttid_stat["time"] = dataTime
        ttid_stat["buy"] = []
        ttid_stat["sell"] = []
        ttid_stat["capacity"] = 0
        for order in orders:
            if order["ttid"] == ttid:
                ttid_stat["capacity"] += order["count"]
                if order["buy"] > 0:
                    ttid_stat["buy"].append(order["buy"] )
                if order["sell"] > 0:
                    ttid_stat["sell"].append(order["sell"] )  
        
        ttid_stat["buystat"] = {}
        if ttid_stat["buy"]:
            ttid_stat["buystat"]["avg"] = st.mean(ttid_stat["buy"])
            ttid_stat["buystat"]["min"] = min(ttid_stat["buy"])
            ttid_stat["buystat"]["max"] = max(ttid_stat["buy"])
            ttid_stat["buystat"]["gap"] = ttid_stat["buystat"]["max"] - ttid_stat["buystat"]["min"]
            ttid_stat["buystat"]["median"] = st.median(ttid_stat["buy"])
            ttid_stat["buystat"]["stdev"] = 0
            if len(ttid_stat["buy"]) >= 2:
                ttid_stat["buystat"]["stdev"] = st.stdev(ttid_stat["buy"], ttid_stat["buystat"]["avg"])
        else:
            ttid_stat["buystat"]["avg"] = 0.0
            ttid_stat["buystat"]["min"] = 0.0
            ttid_stat["buystat"]["max"] = 0.0
            ttid_stat["buystat"]["gap"] = 0.0
            ttid_stat["buystat"]["median"] = 0.0
            ttid_stat["buystat"]["stdev"] = 0.0
            
        ttid_stat["sellstat"] = {}
        if len(ttid_stat["sell"]) >= 2:
            ttid_stat["sellstat"]["avg"] = st.mean(ttid_stat["sell"])
            ttid_stat["sellstat"]["min"] = min(ttid_stat["sell"])
            ttid_stat["sellstat"]["max"] = max(ttid_stat["sell"])
            ttid_stat["sellstat"]["gap"] = ttid_stat["sellstat"]["max"] - ttid_stat["sellstat"]["min"]
            ttid_stat["sellstat"]["median"] = st.median(ttid_stat["sell"])
            ttid_stat["sellstat"]["stdev"] = 0
            if len(ttid_stat["sell"]) >= 2:
                ttid_stat["sellstat"]["stdev"] = st.stdev(ttid_stat["sell"], ttid_stat["sellstat"]["avg"])
        else:
            ttid_stat["sellstat"]["avg"] = 0.0
            ttid_stat["sellstat"]["min"] = 0.0
            ttid_stat["sellstat"]["max"] = 0.0
            ttid_stat["sellstat"]["median"] = 0.0
            ttid_stat["sellstat"]["gap"] = 0.0
            ttid_stat["sellstat"]["stdev"] = 0.0
        
        ttid_summary.append(ttid_stat)
        q = dbquery_insert.replace("\n", " ")
        try:
            cur.execute(q, (ttid_stat["time"],
                                            ttid_stat["ttid"],
                                            str(ttid_stat["buy"]),
                                            str(ttid_stat["sell"]),
                                            ttid_stat["capacity"],
                                            ttid_stat["buystat"]["avg"],
                                            ttid_stat["buystat"]["min"],
                                            ttid_stat["buystat"]["max"],
                                            ttid_stat["buystat"]["median"],
                                            ttid_stat["buystat"]["gap"],
                                            ttid_stat["buystat"]["stdev"],
                                            ttid_stat["sellstat"]["avg"],
                                            ttid_stat["sellstat"]["min"],
                                            ttid_stat["sellstat"]["max"],
                                            ttid_stat["sellstat"]["median"],
                                            ttid_stat["sellstat"]["gap"],
                                            ttid_stat["sellstat"]["stdev"]))
        except Exception as e:
            log.append("db exception %s in query %s" % (q, str(e)))
            return (None, fname, log)
    
    con.commit()
    con.close()
    sync()
    return (dbName, fname, log)

In [29]:
%%time
cleanupTmp()
sync()
full_pathes = [("%s/%s" % (os.getcwd(), f)) for f in data_sources]

def syncExec(pathes):
    parts = []
    counter = 0
    for p in pathes:
        print("%s - try (%d/%d)" % (p, counter, len(full_pathes)))
        parts.append(processSnapshot(p))
        print("%s - done (%d/%d)" % (p, counter, len(full_pathes)))
        sys.stdout.flush()
        counter += 1
    return parts
parts = workers.map_async(processSnapshot, full_pathes).get()
#parts = syncExec(full_pathes)
print("data size %s" % size(asizeof.asizeof(parts)))


clean tmp in /home/ed/repo/github/pyTest/dataSet/haddan/tmp
remove /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1628924401_shops.xml.1631143333068.db
Failed to delete /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1628924401_shops.xml.1631143333068.db. Reason: [Errno 26] Text file busy: '/home/ed/repo/github/pyTest/dataSet/haddan/tmp/1628924401_shops.xml.1631143333068.db'
remove /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1629550801_shops.xml.1631143333073.db
Failed to delete /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1629550801_shops.xml.1631143333073.db. Reason: [Errno 26] Text file busy: '/home/ed/repo/github/pyTest/dataSet/haddan/tmp/1629550801_shops.xml.1631143333073.db'
remove /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1630348201_shops.xml.1631143333069.db
Failed to delete /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1630348201_shops.xml.1631143333069.db. Reason: [Errno 26] Text file busy: '/home/ed/repo/github/pyTest/dataSet/haddan/tmp/1630348201_shops.xml.1631143

data size 897K
CPU times: user 142 ms, sys: 33.6 ms, total: 176 ms
Wall time: 14min 20s


In [30]:
%%time

def mprint(msg: str):
    print(msg)
    sys.stdout.flush()

outDb = "%s/out_%d.db" % (tmpFolder, mstime())
outCon = sqlite3.connect(outDb)
outcur = outCon.cursor()
q = workers["dbquery_cratetable"][0].replace("\n", " ")
outcur.execute(q)
outCon.commit()
for part in parts:
    if not part[0]:
        print("cannot process %s" % part[1])
        for logline in part[2]:
            print(logline)
        break
    dbName = part[0]
    mprint("process %s" % dbName)
    con = sqlite3.connect(dbName)
    cur = con.cursor()
    count = 0
    cur.execute("select * from ttids")
    for row in cur.fetchall():
        q = workers["dbquery_insert"][0].replace("\n", " ")
        outcur.execute(q, row[1::])
        count += 1
        if count > 0 and count % 100 == 0:
            mprint("%d rows merged" % count)
            outCon.commit()
    mprint("done -- %d rows merged" % count)
    con.close()
    
outCon.commit()
outCon.close()

process /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1630956601_shops.xml.1631144992185.db
100 rows merged
200 rows merged
300 rows merged
400 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_1689
500 rows merged
600 rows merged
700 rows merged
800 rows merged
900 rows merged
1000 rows merged
1100 rows merged
1200 rows merged
1300 rows merged
1400 rows merged
1500 rows merged
1600 rows merged
1700 rows merged
1800 rows merged
1900 rows merged
2000 rows merged
2100 rows merged
2200 rows merged
2300 rows merged
2400 rows merged
2500 rows merged
2600 rows merged
2700 rows merged
2800 rows merged
2900 rows merged
3000 rows merged
3100 rows merged
3200 rows merged
3300 rows merged
3400 rows merged
3500 rows merged
3600 rows merged
3700 rows merged
3800 rows merged
3900 rows merged
4000 rows merged
4100 rows merged
4200 rows merged
4300 rows merged
4400 rows merged
4500 rows merged
4600 rows merged
4700 rows merged
4800 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa

2700 rows merged
2800 rows merged
2900 rows merged
3000 rows merged
3100 rows merged
3200 rows merged
3300 rows merged
3400 rows merged
3500 rows merged
done -- 3579 rows merged
process /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1630972801_shops.xml.1631145014156.db
100 rows merged
200 rows merged
300 rows merged
400 rows merged
500 rows merged
600 rows merged
700 rows merged
800 rows merged
900 rows merged
1000 rows merged
1100 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_149053
1200 rows merged
1300 rows merged
1400 rows merged
1500 rows merged
1600 rows merged
1700 rows merged
1800 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_151769
1900 rows merged
2000 rows merged
2100 rows merged
2200 rows merged
2300 rows merged
2400 rows merged
2500 rows merged
2600 rows merged
2700 rows merged
2800 rows merged
2900 rows merged
3000 rows merged
3100 rows merged
3200 rows merged
done -- 3255 rows merged
process /home/ed/repo/github/pyTest/dataSet/had

600 rows merged
700 rows merged
800 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_280976
900 rows merged
1000 rows merged
1100 rows merged
1200 rows merged
1300 rows merged
1400 rows merged
1500 rows merged
1600 rows merged
1700 rows merged
1800 rows merged
1900 rows merged
2000 rows merged
2100 rows merged
2200 rows merged
2300 rows merged
2400 rows merged
2500 rows merged
2600 rows merged
2700 rows merged
2800 rows merged
2900 rows merged
3000 rows merged
3100 rows merged
3200 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_290481
3300 rows merged
3400 rows merged
3500 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_291645
3600 rows merged
3700 rows merged
done -- 3758 rows merged
process /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1630994401_shops.xml.1631145036325.db
100 rows merged
200 rows merged
300 rows merged
400 rows merged
500 rows merged
600 rows merged
700 rows merged
800 rows merged
900 rows merged
1000 rows merge

100 rows merged
200 rows merged
300 rows merged
400 rows merged
500 rows merged
600 rows merged
700 rows merged
800 rows merged
900 rows merged
1000 rows merged
1100 rows merged
1200 rows merged
1300 rows merged
1400 rows merged
1500 rows merged
1600 rows merged
1700 rows merged
1800 rows merged
1900 rows merged
2000 rows merged
2100 rows merged
2200 rows merged
2300 rows merged
2400 rows merged
2500 rows merged
2600 rows merged
2700 rows merged
2800 rows merged
2900 rows merged
3000 rows merged
3100 rows merged
3200 rows merged
3300 rows merged
3400 rows merged
3500 rows merged
3600 rows merged
3700 rows merged
3800 rows merged
3900 rows merged
4000 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_444189
4100 rows merged
4200 rows merged
4300 rows merged
4400 rows merged
4500 rows merged
4600 rows merged
4700 rows merged
done -- 4772 rows merged
process /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1631012401_shops.xml.1631145065468.db
got unknown result: 9c67c6a7-2f

3000 rows merged
3100 rows merged
3200 rows merged
3300 rows merged
3400 rows merged
3500 rows merged
3600 rows merged
3700 rows merged
3800 rows merged
3900 rows merged
4000 rows merged
4100 rows merged
4200 rows merged
4300 rows merged
4400 rows merged
4500 rows merged
4600 rows merged
4700 rows merged
4800 rows merged
4900 rows merged
done -- 4926 rows merged
process /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1631025001_shops.xml.1631145091644.db
100 rows merged
200 rows merged
300 rows merged
400 rows merged
500 rows merged
600 rows merged
700 rows merged
800 rows merged
900 rows merged
1000 rows merged
1100 rows merged
1200 rows merged
1300 rows merged
1400 rows merged
1500 rows merged
1600 rows merged
1700 rows merged
1800 rows merged
1900 rows merged
2000 rows merged
2100 rows merged
2200 rows merged
2300 rows merged
2400 rows merged
2500 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_595217
2600 rows merged
2700 rows merged
got unknown result: 9c67c6a7-2f

...

100 rows merged
200 rows merged
300 rows merged
400 rows merged
500 rows merged
600 rows merged
700 rows merged
800 rows merged
900 rows merged
1000 rows merged
1100 rows merged
1200 rows merged
1300 rows merged
1400 rows merged
1500 rows merged
1600 rows merged
1700 rows merged
1800 rows merged
1900 rows merged
2000 rows merged
2100 rows merged
2200 rows merged
2300 rows merged
2400 rows merged
2500 rows merged
2600 rows merged
2700 rows merged
2800 rows merged
2900 rows merged
3000 rows merged
done -- 3024 rows merged
process /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1630558801_shops.xml.1631145589657.db
100 rows merged
200 rows merged
300 rows merged
400 rows merged
500 rows merged
600 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_3606821
700 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_3607249
800 rows merged
900 rows merged
1000 rows merged
1100 rows merged
1200 rows merged
1300 rows merged
1400 rows merged
1500 rows merged
1600 rows m

3100 rows merged
3200 rows merged
3300 rows merged
3400 rows merged
3500 rows merged
3600 rows merged
3700 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_3733241
3800 rows merged
3900 rows merged
4000 rows merged
4100 rows merged
4200 rows merged
4300 rows merged
4400 rows merged
4500 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_3736229
4600 rows merged
4700 rows merged
4800 rows merged
4900 rows merged
done -- 4965 rows merged
process /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1630573201_shops.xml.1631145612608.db
100 rows merged
200 rows merged
300 rows merged
400 rows merged
500 rows merged
600 rows merged
700 rows merged
800 rows merged
900 rows merged
1000 rows merged
1100 rows merged
1200 rows merged
1300 rows merged
1400 rows merged
1500 rows merged
1600 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_3744649
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_3744813
1700 rows merged
1800 rows merged
1900 rows merg

800 rows merged
900 rows merged
1000 rows merged
1100 rows merged
1200 rows merged
1300 rows merged
1400 rows merged
1500 rows merged
1600 rows merged
1700 rows merged
1800 rows merged
1900 rows merged
2000 rows merged
2100 rows merged
2200 rows merged
2300 rows merged
2400 rows merged
2500 rows merged
2600 rows merged
2700 rows merged
2800 rows merged
2900 rows merged
3000 rows merged
3100 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_3878305
3200 rows merged
3300 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_3879357
3400 rows merged
3500 rows merged
3600 rows merged
3700 rows merged
3800 rows merged
3900 rows merged
4000 rows merged
4100 rows merged
4200 rows merged
4300 rows merged
4400 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_3883473
4500 rows merged
4600 rows merged
4700 rows merged
4800 rows merged
4900 rows merged
done -- 4945 rows merged
process /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1630587601_shops.xml.1

4000 rows merged
4100 rows merged
4200 rows merged
4300 rows merged
done -- 4358 rows merged
process /home/ed/repo/github/pyTest/dataSet/haddan/tmp/1630600201_shops.xml.1631145663181.db
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_4015719
100 rows merged
200 rows merged
300 rows merged
400 rows merged
500 rows merged
600 rows merged
700 rows merged
800 rows merged
900 rows merged
1000 rows merged
got unknown result: 9c67c6a7-2f3101e8fcfa6b6838e8ebee_4019885
1100 rows merged
1200 rows merged
1300 rows merged
1400 rows merged
1500 rows merged
1600 rows merged
1700 rows merged
1800 rows merged
1900 rows merged
2000 rows merged
2100 rows merged
2200 rows merged
2300 rows merged
2400 rows merged
2500 rows merged
2600 rows merged
2700 rows merged
2800 rows merged
2900 rows merged
3000 rows merged
3100 rows merged
3200 rows merged
3300 rows merged
3400 rows merged
3500 rows merged
3600 rows merged
3700 rows merged
3800 rows merged
3900 rows merged
4000 rows merged
4100 rows merged
42

OperationalError: unable to open database file